In [8]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import enchant
import string
import re
import nltk


from nltk import word_tokenize
from nltk.corpus import stopwords
os.chdir("D:/fast_ai/NLP/cap/new_data")
df_k= pd.read_pickle("cleaned_data_setIII.pkl")

In [11]:
## get additional measures for problems in text
## a) find non alphanumeric characters
##b) look at number of stopwords 
## c) how many capital letters are in text
## d) how many non English tokens are there
## e) are there special charcaters like @,!,#...


def text_anomaly(x):
    '''function that gives back if a) all english b) number non alphanumeric c) number stopwords d) number of upper cases e) seldom characters'''    
    
    wt = word_tokenize(x)                      
    non_alphaNUM = len([i for i in  wt if i.isalpha()==False])
    non_English = len([i for i in wt if di.check(i ) == False])
    in_stopwords = len([i for i in wt if i in stop_words])
    upper_letters = len([i for k in wt for i in k if i.isupper()==True ])## generally how many upper we have
    number_special_characters = len([i for i in wt if regex.search(i) != None]) ## special characters.... 
    
    return(non_alphaNUM,non_English,in_stopwords,upper_letters,number_special_characters)
    
def measure_Anomaly(df):
    global di
    di = enchant.Dict("en")
    global stop_words
    stop_words = set(stopwords.words("english"))
    global regex 
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    
    anomal = df["reviews.text"].apply(lambda x: text_anomaly(x))    
    measures = pd.DataFrame(list(zip(*anomal.tolist()))).T
    measures.columns = ["num_nonAlphaN","num_nonEnglish","num_stopwords","num_upperLetters","num_special_characters"]
    return measures 
## we look at ouliers in terms of ratios, i.e. num_of special characters should not be greather than 80% of all characters..
## trying to omit text f.i. "what a cool product!!!!!!!!!!!!!!" because it does not provide information a) and b) is too 
## straight for classifciation 
## though we will use this cleaning step later, when splitting the datasets 

def outlier_detection(df1):
    sub_df =df1.loc[:,["num_nonAlphaN","num_nonEnglish","num_stopwords","num_upperLetters","num_special_characters"]].copy()
    s1 = sub_df.div(df1["text_word_len"],axis=0)
    ix_toolarge = np.where((s1.iloc[:,0]> 0.8) | (s1.iloc[:,1] >0.8))[0]
    ix_too_many = np.where(sub_df["num_special_characters"]/df1["text_character_len"] > 0.3)[0]
    ix_too_short = np.where(df1["text_word_len"] < 10 )[0]
    ix_elims = ix_too_short.tolist() + ix_toolarge.tolist() + ix_too_many.tolist() 

    return set(ix_elims)

In [12]:
oF = measure_Anomaly(df_k)
oF.index= df_k.index
df_k = pd.concat([df_k,oF],axis=1,ignore_index=False)


In [13]:
ix_elim = outlier_detection(df_k)


clean_reviews = df_k.drop(df_k.index[list(ix_elim)])


In [17]:
clean_reviews["reviews.rating"].value_counts(normalize=True)

clean_reviews_1 = clean_reviews.drop_duplicates("reviews.text")

clean_reviews_1.to_pickle("clean_reviews_1.pkl")

np.random.seed(412)

ixx = np.arange(0,clean_reviews_1.shape[0])
clean_reviews_1.index =ixx

ixx_lm = np.random.choice(range(clean_reviews_1.shape[0]),55000,replace=False)
ixx_lm_el = clean_reviews_1.index[ixx_lm]
lm_data_set = clean_reviews_1.loc[ixx_lm_el]

red_set = clean_reviews_1.drop(index=ixx_lm_el)

np.random.seed(123)
ix_train = np.random.choice(range(red_set.shape[0]),33000,replace=False)
data_train = red_set.iloc[ix_train]
data_test = red_set.drop(index= red_set.index[ix_train] )
data_test.shape

data_test["reviews.rating"].value_counts(normalize=True)
data_train["reviews.rating"].value_counts(normalize=True)

#lm_data_set.to_pickle("language_model_data.pkl")
#data_test.to_pickle("test.pkl")
#data_train.to_pickle("train.pkl")


5    0.300970
1    0.270030
3    0.197879
2    0.125545
4    0.105576
Name: reviews.rating, dtype: float64

In [18]:
clean_reviews.shape


(98918, 31)